Installing the required libraries

In [ ]:
!pip install -q torch
!pip install -q langchain
!pip install -q transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q sentence-transformers

Iporting the required libraries

FaceBook AI Similarty Search - FAISS

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering, pipeline
from transformers import AutoModelForQuestionAnswering
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
import pandas as pd
import numpy as np

Loading the dataset

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Text-Generation/RAG/faq_df.csv').iloc[:, 1]
data

0       After you have logged in, you can change your ...
1       You will receive your new ATM PIN by post with...
2       No, for security reasons we send you your ATM ...
3       To register for Autopay: Step 1: Click on the ...
4       Yes, your HDFC Bank Chip Credit card can be us...
                              ...                        
2231    Insta Loan Insta Jumbo Loan Pre-approved Loan ...
2232    Insta Jumbo Loan / Insta Loan: Loan amount wil...
2233    To know the Available Credit Limit in your cre...
2234    Promo Code is an optional field and has to be ...
2235    Login to NetBanking, Click Cards tab > Credit ...
Name: Answer, Length: 2236, dtype: object

In [ ]:
data[:3]

0    After you have logged in, you can change your ...
1    You will receive your new ATM PIN by post with...
2    No, for security reasons we send you your ATM ...
Name: Answer, dtype: object

Splitting the documnet into chunk

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

In [ ]:
docs = text_splitter.create_documents(data)
docs

[Document(page_content='After you have logged in, you can change your password using the \\"Change password\\" option in the top part of the screen. You have to type your current password and the new password you have chosen in their respective boxes.'),
 Document(page_content='You will receive your new ATM PIN by post within 10 days from when your request has been submitted.'),
 Document(page_content='No, for security reasons we send you your ATM PIN only by post.'),
 Document(page_content='To register for Autopay: Step 1: Click on the \\"Autopay register\\" link on the left side margin. Step 2: Select the Credit Card number that you want to register for the Autopay facility and your HDFC Bank account number from which you want your Credit Card payments to be made. Step 3: If you want the full amount of your statement to be paid from your bank account select the \\"Total amount due\\" link, and if you want only the minimum amount due (5% of total amount) to be paid, select \\"Minimum 

In [ ]:
model_path = 'sentence-transformers/all-MiniLM-L6-v2'

In [ ]:
encoder_arg = {'device': 'cpu'}
encoder_test = {'normalize_embeddings': False}

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name=model_path,
                                   model_kwargs=encoder_arg,
                                   encode_kwargs=encoder_test)

In [ ]:
test = "this is test document"

In [ ]:
test_embed = embeddings.embed_query(test)

In [ ]:
test_embed[:3]

[-0.04859999567270279, 0.12207089364528656, -0.06021528318524361]

In [ ]:
db=FAISS.from_documents(docs, embeddings)

In [ ]:
question = "How to check my balance?"

In [ ]:
search_doc = db.similarity_search(question)

In [ ]:
print(search_doc[0].page_content)

You can check the balance on all the active currency wallets on Multicurrency card with the help of Prepaid NetBanking facility. Alternately, you can also contact our PhoneBanking services to check the balance on your Multicurrency card.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForQuestionAnswering.from_pretrained('gpt2')

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
question_answering = pipeline(
    "question-answering",
    model='gpt2',
    tokenizer = tokenizer,
    return_tensors = 'pt'
)

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
retrive_data = db.as_retriever()

In [ ]:
lim = HuggingFacePipeline(pipeline=question_answering)

In [ ]:
retriever = db.as_retriever(search_kwargs={'k': 4})

In [ ]:
qa = RetrievalQA.from_chain_type(llm = lim,
                                 chain_type='refine',
                                 retriever=retrive_data,
                                 return_source_documents=False
                                 )

In [ ]:
ans = qa.run({'query': question})
print(ans['result'])

ValueError: ignored